In [ ]:
from pathlib import Path
import datetime
import os

from wmfdata import hive

# Parameters

The mediawiki_history snapshot for this month must be available.

In [ ]:
last_month = datetime.date.today().replace(day=1) - datetime.timedelta(days=1)

MEDIAWIKI_HISTORY_SNAPSHOT = last_month.strftime("%Y-%m")

metrics_year =  last_month.year
metrics_cur_month = last_month.month

# Update table

## Monthly Pageviews ##

Pageviews data comes from wmf.pageview_hourly for spurious IE views from some countries. Corrected data is stored in intermediate hive table mneisler.pageviews_corrected with data stored back to September 2016. 

List of corrections and timespans that they are applied to:

* IEPKIRAF: since May 20, 2018
* IE7PKIRAF: January 1-May 19, 2018
* IE7PKIRAF & iOS mainpage: Sept 2016-Dec 31, 2017


In [ ]:
update_pv = Path("supplementary/queries/update_pageviews_corrected.hql").read_text().format(
    metrics_year = metrics_year,
    metrics_cur_month = metrics_cur_month
)

In [ ]:
hive.run(["""SET hive.exec.dynamic.partition.mode = 'nonstrict'""",
          update_pv])

Update database access:

In [ ]:
os.system("hdfs dfs -chmod -R o+r /user/hive/warehouse/wmf_product.db/pageviews_corrected")